# Prepare Environment

In [ ]:
from datetime import datetime
import pandas as pd
import plotly
import plotly.express as px
import numpy as np
from statsmodels.tsa.api import VAR
import urllib.request

plotly.offline.init_notebook_mode()

NOW = datetime.now()
TODAY = NOW.date()

print('Aktualisiert:', NOW)

# Initialize Data

In [ ]:
DATA_FILE = 'covid_at_states_timeline.csv'
DATA_URL = "https://covid19-dashboard.ages.at/data/CovidFaelle_Timeline.csv"

STATE_NAMES = ['Burgenland', 'Kärnten', 'Niederösterreich', 
               'Oberösterreich', 'Salzburg', 'Steiermark', 
               'Tirol', 'Vorarlberg', 'Wien']

# TODO: Genauer recherchieren!
EVENTS = {'1. Lockdown': (np.datetime64('2020-03-20'), np.datetime64('2020-04-14'), 
                          'red', 'inside top left'),
          '1. Maskenpflicht': (np.datetime64('2020-03-30'), np.datetime64('2020-05-30'), 
                               'yellow', 'inside bottom left'),
          '2. Maskenpflicht': (np.datetime64('2020-07-25'), np.datetime64(TODAY), 
                               'yellow', 'inside bottom left'),
          'Soft Lockdown': (np.datetime64('2020-11-01'), np.datetime64('2020-11-17'), 
                            'orange', 'inside top left'),
          '2. Lockdown': (np.datetime64('2020-11-17'), np.datetime64(TODAY), 
                          'red', 'inside top left')}

# Only download the data if we don't have it, to avoid
# excessive server access during local development
try:
    with open(DATA_FILE):
        print("Using local data file")
except FileNotFoundError:    
    print("Downloading data")
    urllib.request.urlretrieve(DATA_URL, DATA_FILE)
    
raw_data = pd.read_csv(DATA_FILE, sep=';', parse_dates=[0], infer_datetime_format=True)

In [ ]:
state = raw_data.query("Bundesland == 'Österreich'")
state.insert(0, 'AnzahlFaelle_avg7', state.AnzahlFaelle7Tage / 7)
time = state.Time

# Fallzahlen Österreich (gesamt)

In [ ]:
fig = px.line(state, x='Time', y=["AnzahlFaelle", "AnzahlFaelle_avg7"], log_y=True, title="Fallzahlen")
for name, (begin, end, color, pos) in EVENTS.items():
    fig.add_vrect(x0=begin, x1=end, name=name, fillcolor=color, opacity=0.2, 
                  annotation={'text': name}, annotation_position=pos)
fig.show()

# Wachstumsraten Bundesländer

In [ ]:
states = []
rates = []
for state_name, state_data in raw_data.groupby('Bundesland'):
    x = np.log2(state_data.AnzahlFaelle7Tage)
    rate = 2**np.array(np.diff(x))
    
    rates.append(rate)
    states.append(state_name)
growth = pd.DataFrame({n: r for n, r in zip(states, rates)})

fig = px.line(growth, x=time[1:], y=STATE_NAMES, title='Wachstumsrate')
fig.update_layout(yaxis=dict(range=[0.25, 4]))
fig.show()

In [ ]:
model = VAR(growth[150:][STATE_NAMES])
res = model.fit(1)
res.summary()